In [1]:
# ============================================================
# 01 - Importación de librerías
# ============================================================
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    ElementClickInterceptedException,
    StaleElementReferenceException,
)
import csv, os, re, time, sys, json


# ============================================================
# 02 - Cargar configuración desde JSON
# ============================================================
CONFIG_PATH = "ConfigWBBR.json"

if not os.path.isfile(CONFIG_PATH):
    raise FileNotFoundError(f"[ERROR] No se encontró el archivo {CONFIG_PATH}")

with open(CONFIG_PATH, "r", encoding="utf-8") as f:
    config = json.load(f)

USER = config["USER"]
PWD  = config["PWD"]
LOGIN_URL = config["LOGIN_URL"]
CSPS_URL  = config["CSPS_URL"]


# ============================================================
# 03 - Configuración de archivos de entrada/salida
# ============================================================
INPUT_CSV  = "../InOutWebScrap/InBRWebScrap/partes_entrada.csv"   # Debe tener columna: part_number
OUTPUT_CSV = "../InOutWebScrap/OutBRWebScrap/precios_y_stock.csv"

# Retraso entre consultas para evitar bloqueo
DELAY_BETWEEN_QUERIES = 0.8  # segundos


# ============================================================
# 04 - Configuración del navegador
# ============================================================
opts = Options()
opts.add_argument("--start-maximized")
# opts.add_argument("--headless=new")   # descomentar si quieres correr en background
driver = webdriver.Chrome(options=opts)
W = WebDriverWait(driver, 30)


# ============================================================
# 05 - Funciones utilitarias
# ============================================================
def tnum(s: str) -> str:
    """Convierte un string con número a formato decimal estándar."""
    s = (s or "").replace("\xa0", " ").strip()
    if not any(ch.isdigit() for ch in s):
        return ""
    return s.replace(" ", "").replace(".", "").replace(",", ".")

def tint(s: str) -> int:
    """Extrae solo dígitos de un string y lo convierte a entero."""
    s = re.sub(r"[^\d]", "", s or "")
    return int(s) if s else 0

def tx(xpath: str, wait=30) -> str:
    """Devuelve el texto de un elemento localizado por XPATH."""
    el = WebDriverWait(driver, wait).until(EC.presence_of_element_located((By.XPATH, xpath)))
    return el.text.strip()

def sum_disponibilidad() -> int:
    """Suma la columna de disponibilidad en la tabla de resultados."""
    xp_cells = "//table[.//th[contains(., 'Disponibilidad')]]/tbody//tr[td and count(td)>=2]/td[2]"
    for _ in range(3):
        try:
            cells = driver.find_elements(By.XPATH, xp_cells)
            return sum(tint(c.get_attribute("textContent")) for c in cells)
        except Exception:
            time.sleep(0.3)
    cells = driver.find_elements(By.XPATH, xp_cells)
    return sum(tint(c.get_attribute("textContent")) for c in cells)

def ensure_in_detail_frame():
    """Vuelve siempre al frame 'detail' después de cambios en la página."""
    driver.switch_to.default_content()
    WebDriverWait(driver, 25).until(EC.frame_to_be_available_and_switch_to_it((By.NAME, "detail")))

def open_consulta_general():
    """Abre el menú de 'Consulta General de Referencia' en el portal."""
    driver.switch_to.default_content()
    WebDriverWait(driver, 25).until(EC.frame_to_be_available_and_switch_to_it((By.NAME, "menu")))
    driver.find_element(By.ID, "folderIcon9").click()
    driver.find_element(By.XPATH, "//a[@title='Consulta General de Referencia']").click()
    ensure_in_detail_frame()

def login_and_open_csps():
    """Hace login en el portal CNH y abre CSPS -> Consulta General."""
    # Login
    driver.get(LOGIN_URL)
    W.until(EC.presence_of_element_located((By.ID, "userID"))).send_keys(USER)
    driver.find_element(By.ID, "password").send_keys(PWD)
    driver.find_element(By.ID, "btn-submit").click()

    # Esperar portal y abrir CSPS en nueva pestaña
    WebDriverWait(driver, 40).until(EC.url_contains("my.dlrportal.com"))
    driver.switch_to.new_window("tab")
    driver.get(CSPS_URL)
    WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.TAG_NAME, "frameset")))

    # Menú -> Consulta General de Referencia
    open_consulta_general()


# ============================================================
# 06 - Consultas de partes
# ============================================================
def trigger_consulta_with_enter_on_error(part_input):
    """Intenta hacer clic en 'displayBtn', si falla usa ENTER como fallback."""
    try:
        btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "displayBtn")))
        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", btn)
        btn.click()
        return
    except (ElementClickInterceptedException, TimeoutException):
        try:
            part_input.click()
        except Exception:
            pass
        try:
            driver.execute_script("arguments[0].blur()", part_input)
        except Exception:
            pass
        part_input.send_keys(Keys.ENTER)

def query_part(part_number: str):
    """Consulta un part_number y devuelve los precios y stock en un dict."""
    ensure_in_detail_frame()

    # Campo de referencia
    part = W.until(EC.presence_of_element_located((By.ID, "partNr")))
    part.clear()
    part.send_keys(part_number)
    trigger_consulta_with_enter_on_error(part)

    # Esperar bloque de precios
    WebDriverWait(driver, 40).until(
        EC.presence_of_element_located((By.XPATH, "//td[contains(., 'Máquina Parada Order')]"))
    )

    # Extraer precios
    ref_    = tx("//tbody[.//td[contains(., 'Máquina Parada Order')]]//tr[td[1][contains(., 'Referencia')]]/td[2]")
    pn_mp   = tnum(tx("//tbody[.//td[contains(., 'Máquina Parada Order')]]//tr[td[1][contains(., 'Precio Neto')]]/td[2]"))
    pn_norm = tnum(tx("//tbody[.//td[contains(., 'Máquina Parada Order')]]//tr[td[1][contains(., 'Precio Neto')]]/td[3]"))
    pl_mp   = tnum(tx("//tbody[.//td[contains(., 'Máquina Parada Order')]]//tr[td[1][contains(., 'PriceList')]]/td[2]"))
    pl_norm = tnum(tx("//tbody[.//td[contains(., 'Máquina Parada Order')]]//tr[td[1][contains(., 'PriceList')]]/td[3]"))

    # Extraer disponibilidad
    stock_total = ""
    try:
        hdr = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "//th[contains(., 'Disponibilidad')]"))
        )
        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", hdr)
        time.sleep(0.2)
        stock_total = sum_disponibilidad()
    except TimeoutException:
        stock_total = ""
    except Exception:
        stock_total = ""

    return {
        "Entrada": part_number,
        "Referencia": ref_,
        "PrecioNeto_MaqParada": pn_mp,
        "PrecioNeto_Normal":   pn_norm,
        "PriceList_MaqParada": pl_mp,
        "PriceList_Normal":    pl_norm,
        "StockTotal":          stock_total
    }


# ============================================================
# 07 - Manejo de CSV
# ============================================================
def read_parts_from_csv(path: str):
    """Lee los part numbers desde el CSV de entrada."""
    parts = []
    with open(path, "r", encoding="utf-8-sig", newline="") as f:
        reader = csv.reader(f)
        rows = list(reader)
        if not rows:
            return parts
        header = [h.strip().lower() for h in rows[0]]
        start_idx = 1
        col_idx = 0
        if any(h for h in header):
            if "part_number" in header:
                col_idx = header.index("part_number")
            else:
                col_idx = 0
        else:
            start_idx = 0
            col_idx = 0

        for r in rows[start_idx:]:
            if not r:
                continue
            val = (r[col_idx] or "").strip()
            if val:
                parts.append(val)
    return parts

def append_output_row(path, row_dict, write_header_if_needed=True):
    """Agrega resultados al CSV de salida."""
    write_header = write_header_if_needed and (not os.path.isfile(path) or os.path.getsize(path) == 0)
    with open(path, "a", newline="", encoding="utf-8-sig") as f:
        w = csv.writer(f)
        if write_header:
            w.writerow([
                "Entrada", "Referencia",
                "PrecioNeto_MaqParada", "PrecioNeto_Normal",
                "PriceList_MaqParada",  "PriceList_Normal",
                "StockTotal",
                "Status", "Mensaje"
            ])
        w.writerow([
            row_dict.get("Entrada", ""),
            row_dict.get("Referencia", ""),
            row_dict.get("PrecioNeto_MaqParada", ""),
            row_dict.get("PrecioNeto_Normal", ""),
            row_dict.get("PriceList_MaqParada", ""),
            row_dict.get("PriceList_Normal", ""),
            row_dict.get("StockTotal", ""),
            row_dict.get("Status", ""),
            row_dict.get("Mensaje", "")
        ])


# ============================================================
# 08 - Flujo principal
# ============================================================
try:
    login_and_open_csps()

    parts = read_parts_from_csv(INPUT_CSV)
    if not parts:
        print(f"[ERROR] El CSV '{INPUT_CSV}' no tiene datos.")
        sys.exit(1)
    print(f"[INFO] Partes a consultar: {len(parts)}")

    for i, p in enumerate(parts, 1):
        max_attempts = 2
        last_err = None
        for attempt in range(1, max_attempts + 1):
            try:
                data = query_part(p)
                data["Status"]  = "OK"
                data["Mensaje"] = ""
                append_output_row(OUTPUT_CSV, data)
                print(f"[{i}/{len(parts)}] OK -> {p}  Ref={data.get('Referencia','')}  Stock={data.get('StockTotal','')}")
                break
            except (TimeoutException, StaleElementReferenceException) as te:
                last_err = te
                try:
                    open_consulta_general()
                except Exception:
                    pass
                time.sleep(0.5)
            except Exception as e:
                last_err = e
                time.sleep(0.5)
        else:
            append_output_row(OUTPUT_CSV, {
                "Entrada": p,
                "Referencia": "",
                "PrecioNeto_MaqParada": "",
                "PrecioNeto_Normal": "",
                "PriceList_MaqParada": "",
                "PriceList_Normal": "",
                "StockTotal": "",
                "Status": "ERROR",
                "Mensaje": str(last_err)[:250]
            })
            print(f"[{i}/{len(parts)}] ERROR -> {p}: {last_err}")

        time.sleep(DELAY_BETWEEN_QUERIES)

    print(f"\n[FIN] CSV actualizado: {OUTPUT_CSV}")

except Exception as e:
    print("[FATAL]", e)

finally:
    try:
        input("ENTER para cerrar...")
    except:
        pass
    driver.quit()


[INFO] Partes a consultar: 13
[1/13] OK -> 5801781949  Ref=5801781949  Stock=127
[2/13] OK -> 47133709  Ref=47133709  Stock=0
[3/13] OK -> 504127327  Ref=504127327  Stock=807
[4/13] OK -> 5801781935  Ref=5801781935  Stock=108
[5/13] OK -> 500042644  Ref=500042644  Stock=250
[6/13] OK -> 500042643  Ref=500042643  Stock=261
[7/13] OK -> 5801781949  Ref=5801781949  Stock=127
[8/13] OK -> 504092109  Ref=504092109  Stock=1501
[9/13] OK -> 48133712  Ref=48133712  Stock=50
[10/13] OK -> 5802486932  Ref=5802486932  Stock=318
[11/13] OK -> 84455696  Ref=84455696  Stock=296
[12/13] OK -> 98473706  Ref=98473706  Stock=2589
[13/13] OK -> 1930181  Ref=1930181  Stock=896

[FIN] CSV actualizado: ../InOutWebScrap/OutBRWebScrap/precios_y_stock.csv
